## Trabalhando com Big Data - 25 Milhões de Registros - Usando Modin
fonte: https://grouplens.org/datasets/movielens/

In [1]:
#import pandas as pd

## Scale your pandas workflows by changing one line of code <br>
https://github.com/modin-project/modin

In [2]:
# pip install modin
# conda install -c conda-forge modin-all

In [3]:
# import pandas as pd

from distributed import Client
client = Client(n_workers=6)
# Modin will connect to the Dask Client
import modin.pandas as pd


#import modin.pandas as pd
#from dask.distributed import Client

#client = Client()

In [4]:
!pwd

/Users/valencar/Dropbox/AULAS-REMOTAS/Alunos-UEPB-TopicosEspeciaisEmBancoDeDados/Python-Para-Analise-de-Dados/MANHA


In [5]:
!ls -ilah /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m

total 4645512
8852112067 drwxr-xr-x@ 15 valencar  staff   480B Feb  1 18:51 .
8753341664 drwxr-xr-x@ 31 valencar  staff   992B Dec 21 05:24 ..
8852135013 -rw-r--r--@  1 valencar  staff   6.0K Dec 21 16:32 .DS_Store
8852112072 -rw-r--r--@  1 valencar  staff    10K Oct 17  2016 README.txt
8852115885 -rw-r--r--@  1 valencar  staff   1.1M Dec 20 17:50 filmes-ratings.csv
8852133601 -rw-r--r--@  1 valencar  staff    46K Dec 20 21:38 filmes2.csv
8859960005 -rw-r-----@  1 valencar  staff   1.1G Dec 20 22:20 filmes4.csv
8852142504 -rw-r--r--@  1 valencar  staff   287M Dec 21 05:20 filmes4.csv.zip
8852112073 -rw-r--r--@  1 valencar  staff   309M Oct 17  2016 genome-scores.csv
8852112070 -rw-r--r--@  1 valencar  staff    18K Oct 17  2016 genome-tags.csv
8852112068 -rw-r--r--@  1 valencar  staff   557K Oct 17  2016 links.csv
8852445227 -rw-r--r--@  1 valencar  staff   2.3K Dec 24 23:22 ml-20m.sql
8852112074 -rw-r--r--@  1 valencar  staff   1.3M Mar 31  2015 movies.csv
8852112071 -rw-r--r--@  1 val

In [6]:
!head /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/movies.csv

In [7]:
#!unzip /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/filmes4.csv.zip 

In [8]:
#!mv filmes4.csv /Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/

In [9]:
url = '/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m' + '/' + 'ratings.csv'
url

'/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/ratings.csv'

In [10]:
%time

from dask import dataframe as dd


dfratings = dd.read_csv(url)
print('total de registros: {:,d}'.format(len(dfratings)))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


total de registros: 20,000,263


In [11]:
dfratings.head()

userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580

In [12]:
url = '/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m' + '/' + 'movies.csv'
url

'/Users/valencar/Dropbox/AULAS-REMOTAS/BD-ESTATISTICA-MANHA/datasets/ml-20m/movies.csv'

In [13]:
%%time

dfmovies = dd.read_csv(url)
print('total de registros: {:,d}'.format(len(dfmovies)))

total de registros: 27,278
CPU times: user 17.5 ms, sys: 4.2 ms, total: 21.7 ms
Wall time: 76 ms


In [14]:
dfmovies.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [15]:
dfratings.head()

userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580

# Join - inner join

In [16]:
%%time

dffilmes =  dd.merge(dfmovies, dfratings, left_on='movieId', right_on='movieId')
print('total de registros: {:,d}'.format(len(dffilmes)))
dffilmes.head()

total de registros: 20,000,263
CPU times: user 360 ms, sys: 55.4 ms, total: 415 ms
Wall time: 6.14 s


movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   userId  rating   timestamp  
0       3     4.0   944919407  
1       6     5.0   858275452  
2       8     4.0   833981871  
3      10     4.0   943497887  
4      11     4.5  1230858821

In [17]:
dffilmes.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 6 entries, movieId to timestamp
dtypes: object(2), float64(1), int64(3)

In [18]:
%%time

dffilmes.compute().sort_values(by='rating', ascending=False)[:30] 

CPU times: user 7.23 s, sys: 2.41 s, total: 9.64 s
Wall time: 16.2 s


movieId                                              title  \
510544       608                                       Fargo (1996)   
1695061     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
869849      1285                                    Heathers (1989)   
1695071     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
1695070     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
1695068     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
1695067     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
1965718     6281                                 Phone Booth (2002)   
1695063     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
1971261     6377                                Finding Nemo (2003)   
1114780     1968                         Breakfast Club, The (1985)   
1965714     6281                                 Phone Booth (2002)   
1982264     7256                           Touching the Void (2003)   
869843      1285                                    Heathers (1989)   
869842      1285                                    Heathers (1989)   
1114784     1968                         Breakfast Club, The (1985)   
1114785     1968                         Breakfast Club, The (1985)   
1695053     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
1982177     7254                        The Butterfly Effect (2004)   
1982175     7254                        The Butterfly Effect (2004)   
869837      1285                                    Heathers (1989)   
1982173     7254                        The Butterfly Effect (2004)   
869850      1285                                    Heathers (1989)   
1695074     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
1695047     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
1114768     1968                         Breakfast Club, The (1985)   
2211991    30793           Charlie and the Chocolate Factory (2005)   
1695102     3996  Crouching Tiger, Hidden Dragon (Wo hu cang lon...   
1971239     6377                                Finding Nemo (2003)   
1982298     7256                           Touching the Void (2003)   

                                         genres  userId  rating   timestamp  
510544              Comedy|Crime|Drama|Thriller   64895     5.0   835784283  
1695061                    Action|Drama|Romance   20314     5.0  1072020681  
869849                                   Comedy   82262     5.0  1085422722  
1695071                    Action|Drama|Romance   20342     5.0   983922370  
1695070                    Action|Drama|Romance   20340     5.0  1375431534  
1695068                    Action|Drama|Romance   20332     5.0  1234473138  
1695067                    Action|Drama|Romance   20331     5.0  1010804653  
1965718                          Drama|Thriller   40292     5.0  1074444269  
1695063                    Action|Drama|Romance   20322     5.0  1224530134  
1971261     Adventure|Animation|Children|Comedy   51113     5.0  1207345026  
1114780                            Comedy|Drama   36009     5.0  1025618746  
1965714                          Drama|Thriller   40174     5.0  1253136560  
1982264                   Adventure|Documentary  100937     5.0  1148260057  
869843                                   Comedy   82211     5.0   975728618  
869842                                   Comedy   82161     5.0   991699639  
1114784                            Comedy|Drama   36036     5.0  1376605406  
1114785                            Comedy|Drama   36045     5.0  1257634389  
1695053                    Action|Drama|Romance   20278     5.0  1198102644  
1982177                   Drama|Sci-Fi|Thriller  116775     5.0  1314706620  
1982175                   Drama|Sci-Fi|Thriller  116754     5.0  1278382615  
869837                                   Comedy   82082     5.0   957652959  
1982173                   Drama|Sci-Fi|Thriller  116706     5.0  1175821747  
869850       

In [19]:
%%time

dfdrama_acao = dffilmes[ dffilmes.genres.str.contains('Drama') |  dffilmes.genres.str.contains('Action') ]
print('total de registros: {:,d}'.format(len(dfdrama_acao)))
dfdrama_acao.head()

total de registros: 13,088,050
CPU times: user 824 ms, sys: 143 ms, total: 967 ms
Wall time: 16.9 s


movieId                     title                genres  userId  \
10373        4  Waiting to Exhale (1995)  Comedy|Drama|Romance      41   
10374        4  Waiting to Exhale (1995)  Comedy|Drama|Romance      61   
10375        4  Waiting to Exhale (1995)  Comedy|Drama|Romance     156   
10376        4  Waiting to Exhale (1995)  Comedy|Drama|Romance     158   
10377        4  Waiting to Exhale (1995)  Comedy|Drama|Romance     295   

       rating   timestamp  
10373     2.0   951694198  
10374     3.0  1197068409  
10375     3.0  1038801803  
10376     3.0   842735262  
10377     3.0   863971097

## Criando colunas Genero - individuais

In [20]:
genres = 'Comedy|Drama|Romance'
generos = genres.split('|')
generos

['Comedy', 'Drama', 'Romance']

In [21]:
%%time

for index, row in dffilmes.iterrows(): 
    print(row['genres'])
    generos = row['genres'].split('|')
    print(generos)
    break

Adventure|Animation|Children|Comedy|Fantasy
['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']
CPU times: user 683 ms, sys: 236 ms, total: 919 ms
Wall time: 2.28 s


In [22]:
%%time

set_generos = set()

for index, row in dffilmes.iterrows(): 
    print(row['genres'])
    generos = row['genres'].split('|')
    set_generos.update(generos)
    print(set_generos)
    break

Adventure|Animation|Children|Comedy|Fantasy
{'Comedy', 'Animation', 'Children', 'Adventure', 'Fantasy'}
CPU times: user 895 ms, sys: 365 ms, total: 1.26 s
Wall time: 2.78 s


In [23]:
%%time

set_generos = set()

for index, row in dffilmes.iterrows(): 
    generos = row['genres'].split('|')
    set_generos.update(generos)
    
generos = list(set_generos)
generos

CPU times: user 11min 49s, sys: 13.9 s, total: 12min 3s
Wall time: 12min 32s


['Animation',
 'Documentary',
 'Film-Noir',
 'Western',
 'Crime',
 'Romance',
 'Musical',
 'Fantasy',
 'Thriller',
 'Children',
 'Horror',
 'War',
 'Action',
 'IMAX',
 '(no genres listed)',
 'Comedy',
 'Drama',
 'Sci-Fi',
 'Mystery',
 'Adventure']

In [24]:
print(generos, len(generos))

['Animation', 'Documentary', 'Film-Noir', 'Western', 'Crime', 'Romance', 'Musical', 'Fantasy', 'Thriller', 'Children', 'Horror', 'War', 'Action', 'IMAX', '(no genres listed)', 'Comedy', 'Drama', 'Sci-Fi', 'Mystery', 'Adventure'] 20


In [25]:
# generos = ['Crime', 'Western', 'Thriller', 'War', 'Sci-Fi', 'Children', 
#             'Documentary', '(no genres listed)', 'Romance', 'Adventure', 'Mystery',
#             'Comedy', 'IMAX', 'Animation', 'Action', 'Horror', 'Film-Noir', 'Fantasy',
#         'Drama', 'Musical'] 

In [26]:
%%time

for genero in generos:
    dffilmes[genero] = False
    
dffilmes.head()

CPU times: user 122 ms, sys: 12.8 ms, total: 135 ms
Wall time: 1.11 s


movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   userId  rating   timestamp  Animation  Documentary  Film-Noir  Western  \
0       3     4.0   944919407      False        False      False    False   
1       6     5.0   858275452      False        False      False    False   
2       8     4.0   833981871      False        False      False    False   
3      10     4.0   943497887      False        False      False    False   
4      11     4.5  1230858821      False        False      False    False   

   ...  Horror    War  Action   IMAX  (no genres listed)  Comedy  Drama  \
0  ...   False  False   False  False               False   False  False   
1  ...   False  False   False  False               False   False  False   
2  ...   False  False   False  False               False   False  False   
3  ...   False  False   False  False               False   False  False   
4  ...   False  False   False  False               False   False  False   

   Sci-Fi  Mystery  Adventure  
0   False    False      False  
1   False    False      False  
2   False    False      False  
3   False    False      False  
4   False    False      False  

[5 rows x 26 columns]

In [ ]:
%%time 

for index, row in dffilmes.iterrows(): 
    generos = row['genres'].split('|')
    for genero in generos:
        #print(genero)
     
        dffilmes.compute().at[index, genero] = True

        #print(dffilmes.iloc[index][genero])
        #break
    #break  
    
dffilmes.head()

CPU times: user 31min 32s, sys: 32.6 s, total: 32min 5s
Wall time: 34min 12s

In [ ]:
dffilmes.to_csv('filmes-2022-Modin.csv', index=False)
!ls -ilah *.csv

In [ ]:
dffilmes.Action.value_counts()

In [ ]:
%time
#df2 = pd.get_dummies(dfdrama_acao, columns=['genres'])
#df2.head()

In [ ]:
print(df2.columns, len(df2.columns))

In [ ]:
dfo = dffilmes.rating.value_counts()
dfo

In [ ]:
type(dfo)

In [ ]:
df2 = dfo.to_frame().reset_index()
df2.columns = ['rating', 'quantidade']

In [ ]:
df2

In [ ]:
df2['percent'] = df2['quantidade'] / df2['quantidade'].sum() * 100
df2

In [ ]:
# formatar a saída

df3 = df2.style.format({
    "rating": "{:.1f}",
    "quantidade": "{:,d}",
    "percent": "{:.0f}"
})



df3

In [ ]:
df4 = df2[ df2.percent >= 7]
df4

In [ ]:
df4.percent.sum()

In [ ]:
print('{:,d}'.format(df4.quantidade.sum()))

In [ ]:
df5 = df2[ df2.percent < 7]
df5

In [ ]:
print('{:,d}'.format(df5.quantidade.sum()))

In [ ]:
df5.percent.sum()

In [ ]:
df5

In [ ]:
row = {'rating': 'outros', 'quantidade': df5.quantidade.sum(), 'percent': df5.percent.sum()  }
row

In [ ]:
df6 = df4.append(row, ignore_index=True)
df6

In [ ]:
# formatar a saída

dfs = df6.style.format({
    "rating": "{}",
    "quantidade": "{:,d}",
    "percent": "{:.0f}"
})

dfs

In [ ]:
df6.to_csv('ratings_saida.csv', index=False)

In [ ]:
!cat ratings_saida.csv

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (9,9)

#define data
data = df6.percent
labels = df6.rating

#define Seaborn color palette to use
colors = sns.color_palette('pastel')[0:7]

#create pie chart
#print('Filmes por avaliação (1-5)')


fig = plt.figure()

ax = fig.add_axes([0, 0, .8, .8], aspect=1)

ax.set_title('Filmes por avaliação (1-5)')
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')


plt.show()

# 1 Bilhão de Registros

In [ ]:
!ls ~/Downloads/ml-20mx16x32

In [ ]:
pwd

In [ ]:
#!pip install -q npzviewer

In [ ]:
%%time
import numpy as np

url  = '/Users/valencar/Downloads/ml-20mx16x32/'
n = np.load(url + 'testx16x32_0.npz')

In [ ]:
 n.files

In [ ]:
n['arr_0'].shape

In [ ]:
n['arr_0'][20]

In [ ]:
%%time
import pandas as pd

url = 

df = pd.read_csv(url)
df.head